In [1]:
import casadi as ca
import numpy as np 
import json 
import pandas as pd 

%load_ext autoreload
%autoreload 2

In [2]:
with open('mpc_coefficients.json', 'r') as json_file:
    model_coefficients = json.load(json_file)

In [3]:
import casadi as ca
import numpy as np


def setup_OptiProblem(N,zs,Ts)->ca.Function:
    # Create an Opti instance
    opti = ca.Opti()

    opti.solver("ipopt")

    ### Parameters
    # #============= Tank Model ==================
    A = 18
    # #=========== Parameters of the static models
    B1_pressure = np.array(model_coefficients["a_pressure"])
    B2_pressure = np.array(model_coefficients["b_pressure"])
    C_pressure =  np.array(model_coefficients["c_pressure"])

    #=========== Parameters of the ARX model
    # array of params
    A_power = np.array(model_coefficients["A_power"])
    B_power = np.array(model_coefficients["B_power"])
    C_power = np.array(model_coefficients["C_power"])

    #=========== Desired value of y  ascending index: older data at 0, last measured value (t-1)
    Qout_meas = opti.parameter(zs) 
    h_meas = opti.parameter(zs) 
    P_meas = opti.parameter(zs)
    Qin_est = opti.parameter(N+zs)
    w_meas =  opti.parameter(3,zs)
    h_ref = opti.parameter(1)
    E_meas = opti.parameter(3,zs)
    trigger = opti.parameter(3) # Trigger constant over entire horizon

    #=========== Declare Symbolic Variables
    Qout = opti.variable(N+zs)
    E = opti.variable(3,N+zs)
    P = opti.variable(N+zs)
    w = opti.variable(3,N+zs)
    h = opti.variable(N+zs)

    #============= Slack Variables
    s_h = opti.variable(N+zs)
    s_P = opti.variable(N+zs)

    #=========== Objective function
    objective = 0

    for t in range(zs, N+zs):
        objective += 10*(h[t]-h_ref)**2  + 0.5*((w[:,t]-w[:,t-1]).T @ (w[:,t]-w[:,t-1]))+1000*s_P[t]**2 + 1000*s_h[t]**2 + 0.5*(ca.if_else(trigger[0] > 0, 0,w[0,t]) + ca.if_else(trigger[1] > 0, 0,w[1,t]) +ca.if_else(trigger[2] > 0, 0,w[2,t]))

    opti.minimize(objective)   


    # ARX model constraints
    for t in range(zs, N+zs):

        opti.subject_to(Qout[t] ==ca.if_else( 
                                    w[0,t-1] <= model_coefficients["speed_breakpoint"],
                                    3.2216 + 0.08378681 * w[0,t-1],  # First segment equation
                                    3.22 + 0.083 * 600 + 0.8371 * (w[0,t-1] - 600),  # Second segment equation, ensuring continuity at the breakpoint
                                    True) 
                                + ca.if_else(
                                    w[1,t-1] <= 600,
                                    3.2216 + 0.08378681 * w[1,t-1],  
                                    3.22 + 0.083 * 600 + 0.8371 * (w[1,t-1] - 600),  
                                    True)
                                + ca.if_else(
                                    w[2,t-1] <= 600,
                                    3.2216 + 0.08378681 * w[2,t-1],
                                    3.22 + 0.083 * 600 + 0.8371 * (w[2,t-1] - 600),
                                    True))


        opti.subject_to(E[:,t] == A_power @ ca.vcat([E[:,t-1],E[:,t-2]]) + B_power @ w[:,t-1] + C_power)

        opti.subject_to(P[t] == B1_pressure @ Qout[t-1] + B2_pressure @ Qout[t-1]**2 + C_pressure) 
        
        opti.subject_to(h[t] == h[t-1] + Ts/3600*(Qin_est[t-1]-Qout[t-1])/A)


    # Additional constraints (e.g., on control input)
    for t in range(zs, N+zs):
        opti.subject_to(w[:,t] >= 0)  # Lower bound on control input
        opti.subject_to(w[:,t] <= 1500)
        opti.subject_to(h[t] <= (200 + s_h[t]))
        opti.subject_to(h[t] >= (120 - s_h[t]))
        opti.subject_to(P[t] <= 1)
        opti.subject_to(P[t] >= 0)
        opti.subject_to(s_h[t] >= 0)
        opti.subject_to(s_P[t] >= 0)


    # Initial conditions
    #opti.subject_to(Qout[0:zs]   == Qout_meas)
    opti.subject_to(h[0:zs]      == h_meas)
    opti.subject_to(w[:,0:zs]    == w_meas)  
    opti.subject_to(P[0:zs]      == P_meas)
    opti.subject_to(E[:,0:zs]    == E_meas)
    
    inputs = [Qin_est,Qout_meas,h_meas,w_meas,E_meas,P_meas,h_ref, trigger]
    outputs = [w,Qout,h,E,P]
    mpc_controller = opti.to_function("mpc_controller", inputs, outputs)
    
    return mpc_controller

In [4]:
df = pd.read_parquet("../data/sym_data/sym_df_5s_res_withPower.parquet")
df = df["2023-05-23 20:00:00":"2023-06-13 08:00:00"]

In [5]:
from funcs import MPCVariableManager

In [6]:
zs = 3 # Number of lags
N = 24 #horion 
Ts = 5 #sampling time
trigger_k = [1, 0, 0]
Qin_k = df["inflow"].values
h_ref = 120

mpc_data_buffer = MPCVariableManager()
mpc_data_buffer.initialize(df, zs)

mpc_controller = setup_OptiProblem(N,zs,Ts)

In [12]:

for k in range(0,len(Qin_k[:500])):
    
    ### SOLVE MPC 
    sol_w,sol_Qout,sol_h,sol_E,sol_P = mpc_controller(Qin_k[k+zs:k+zs+N+zs],
                                                    mpc_data_buffer.Qout_hist[-zs:],
                                                    mpc_data_buffer.h_hist[-zs:],
                                                    mpc_data_buffer.w_hist[:,-zs:],
                                                    mpc_data_buffer.E_hist[:,-zs:],
                                                    mpc_data_buffer.P_hist[-zs:],
                                                    h_ref, 
                                                    trigger_k)
    
    ### Update data buffer
    mpc_data_buffer.update(sol_w,sol_Qout,sol_h,sol_E,sol_P)
    
results_df = mpc_data_buffer.create_results_dataframe()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     2832
Number of nonzeros in inequality constraint Jacobian.:      336
Number of nonzeros in Lagrangian Hessian.............:     3417

Total number of variables............................:      297
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      168
Total number of inequality constraints...............:      288
        inequality constraints with only lower bounds:      144
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      144

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.4560000e+06 1.50e+03 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [14]:
results_df

,w1,w2,w3,Qout,h,P,E1,E2,E3
0,810.000000,0.0,0.0,174.000000,152.000000,0.554200,38.237871,0.0,0.002654
1,810.000000,0.0,0.0,174.800000,152.200000,0.554800,38.237871,0.0,0.002654
2,810.000000,0.0,0.0,173.600000,153.000000,0.559400,38.237871,0.0,0.002654
3,820.681728,0.0,0.0,235.254200,152.799257,1.000000,38.237871,0.0,0.000000
4,831.294338,0.0,0.0,244.195875,152.598500,1.000000,38.612981,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
608,1417.896566,0.0,0.0,744.263449,120.719422,0.567013,66.946500,0.0,0.000000
609,1417.729281,0.0,0.0,744.124415,120.723056,0.566348,66.938681,0.0,0.000000
610,1417.560710,0.0,0.0,743.984381,120.727097,0.565808,66.930808,0.0,0.000000
611,1417.390898,0.0,0.0,743.843270,120.731229,0.565712,66.922877,0.0,0.000000


In [13]:
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
 
 
fig = FigureWidgetResampler(go.Figure())
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.add_trace(go.Scattergl(name=r'Speed1 rpm', showlegend=True), hf_x=results_df.index, hf_y=results_df['w1'])
fig.add_trace(go.Scattergl(name=r'Speed2 rpm', showlegend=True), hf_x=results_df.index, hf_y=results_df['w2'])
fig.add_trace(go.Scattergl(name=r'Speed3 rpm', showlegend=True), hf_x=results_df.index, hf_y=results_df['w3'])
fig.add_trace(go.Scattergl(name=r'Outflow', showlegend=True), hf_x=results_df.index, hf_y=results_df['Qout'])
fig.add_trace(go.Scattergl(name=r'Height', showlegend=True), hf_x=results_df.index, hf_y=results_df['h'])
fig.add_trace(go.Scattergl(name=r'Pressure', showlegend=True), hf_x=results_df.index, hf_y=results_df['P'])
fig.add_trace(go.Scattergl(name=r'Power P1', showlegend=True), hf_x=results_df.index, hf_y=results_df['E1'])
fig.add_trace(go.Scattergl(name=r'Power P2', showlegend=True), hf_x=results_df.index, hf_y=results_df['E2'])
fig.add_trace(go.Scattergl(name=r'Power P2', showlegend=True), hf_x=results_df.index, hf_y=results_df['E3'])


fig.update_layout(height=400, template="plotly_dark")
display(fig)

FigureWidgetResampler({
    'data': [{'name': 'Speed1 rpm',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'd3eda508-24fb-47c4-8c1f-6dcf04284ed9',
              'x': array([  0,   1,   2, ..., 610, 611, 612]),
              'y': array([ 810.        ,  810.        ,  810.        , ..., 1417.56071008,
                          1417.39089786, 1417.2200212 ])},
             {'name': 'Speed2 rpm',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'e7d43fc3-5daa-4ff9-9487-45d12d8f57b3',
              'x': array([  0,   1,   2, ..., 610, 611, 612]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'name': 'Speed3 rpm',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '9cff8df0-e2fe-4041-9064-4dfb7efc3860',
              'x': array([  0,   1,   2, ..., 610, 611, 612]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'name': '